<a href="https://colab.research.google.com/github/azamatarmanuly99/docs/blob/main/notebooks/06.04-Protein-Design-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Protein Design with a Resfile and FastRelax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [HBNet Before Design](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.05-HBNet-Before-Design.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.04-Protein-Design-2.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Protein Design 2

Keywords: FastDesign, FastRelax, ResidueSelector, TaskFactory, TaskOperation, pose_from_rcsb, get_residues_from_subset, ResidueNameSelector, ChainSelector, ResiduePropertySelector, NotResidueSelector, AndResidueSelector, RestrictToRepackingRLT, RestrictAbsentCanonicalAASRLT, NoRepackDisulfides, OperateOnResidueSubset

## Overview

Here, we will expand upon our knowledge of ResidueSelectors and TaskOperations and how to use them.  We can once again use FastRelax or FastDesign to do design.  Since we don't need any additional tools offered by FastDesign, we will use FastRelax.  I like to call it RelaxedDesign. :)

*Warning*: This notebook uses `pyrosetta.distributed.viewer` code, which runs in `jupyter notebook` and might not run if you're using `jupyterlab`.

In [1]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()


Mounted at /content/google_drive
Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin//wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2024.1+release.00b79147e63-cp310-cp310-linux_x86_64.whl


PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.01+release.00b79147e63be743438188f93a3f069ca75106d6 2023-12-25T16:35:48] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r366 2024.01+release.00b79147e63 00b79147e63be743438188f93a3f069ca75106d6 http://www.pyrosetta.org 2023-12-25T16:35:48
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.10/dist-packages/pyrosetta/database
basic.random.init_ra

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

In [2]:
import logging
logging.basicConfig(level=logging.INFO)
import os
import pyrosetta
import pyrosetta.toolbox
import site

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Initialize PyRosetta

In [3]:
pyrosetta.init("-ignore_unrecognized_res 1 -ex1 -ex2aro")

PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.01+release.00b79147e63be743438188f93a3f069ca75106d6 2023-12-25T16:35:48] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r366 2024.01+release.00b79147e63 00b79147e63be743438188f93a3f069ca75106d6 http://www.pyrosetta.org 2023-12-25T16:35:48
core.init: command: PyRosetta -ignore_unrecognized_res 1 -ex1 -ex2aro -database /usr/local/lib/python3.10/dist-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=1810679065 seed_offset=0 real_seed=1810679065
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=1810679065 RG_type=mt19937


 For this tutorial, let's again use the native protein crambin from PDB ID 1AB1 (http://www.rcsb.org/structure/1AB1)
 Setup the input pose and scorefunction:

In [4]:
start_pose = pyrosetta.toolbox.rcsb.pose_from_rcsb("1AB1", ATOM=True, CRYS=False)
pose = start_pose.clone()
scorefxn = pyrosetta.create_score_function("ref2015_cart.wts")

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.18675 seconds.
core.import_pose.import_pose: File '1AB1.clean.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 3 atoms at position 1 in file 1AB1.clean.pdb. Best match rsd_type:  THR:NtermProteinFull
core.conformation.Conformation: Found disulfide between residues 3 40
core.conformation.Conformation: Found disulfide between residues 4 32
core.conformation.Conformation: Found disulfide between residues 16 26
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating ener

 To list the cysteine residues in crambin, now we use ResidueSelectors:
 Note difference in residue names "CYS:disulfide" vs. "CYS"

 Note that we could also use the `ResiduePropertySelector` and use the `DISULFIDE_BONDED` property to grab these.

In [5]:
disulfide_res = pyrosetta.rosetta.core.select.residue_selector.ResidueNameSelector("CYS:disulfide")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)))

vector1_unsigned_long[3, 4, 16, 26, 32, 40]


 Inspect the starting pose using the `PyMolMover` or `dump_pdb()`. By default, disulfide bonds are not drawn even if they exist in PyRosetta:

 You can also use this shortcut to view a pose in `py3Dmol` which recognizes disulfide bonds in PyRosetta:

Run one of the following, whether you git cloned the pyrosetta_scripts repository or downloaded visualization.py from the Google Drive

`%run ~/pyrosetta_scripts/pilot/apps/klimaj/py3Dmol/visualization.py`
`%run ./visualization.py`

## Design strategy

Design away the cysteine residues (i.e. disulfide bonds) using ResidueSelectors and TaskOperations, allowing all side-chains to re-pack and all backbone and side-chain torsions to minimize using the `FastDesign` mover.

 Note: because we did not include the `-detect_disulfide 0` command line flag in `pyrosetta.init()`, in order to mutate disulfide bonded cysteine residues using packing steps in the `FastDesign` mover, first we need to break the disulfide bonds:

In [6]:
for i in pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)):
    for j in pyrosetta.rosetta.core.select.get_residues_from_subset(disulfide_res.apply(pose)):
        if pyrosetta.rosetta.core.conformation.is_disulfide_bond(pose.conformation(), i, j):
            pyrosetta.rosetta.core.conformation.break_disulfide(pose.conformation(), i, j)

 Now we can declare a `ResidueSelector` for cysteine residues:

In [7]:
cys_res = pyrosetta.rosetta.core.select.residue_selector.ResidueNameSelector("CYS")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(cys_res.apply(pose)))

vector1_unsigned_long[3, 4, 16, 26, 32, 40]


 Note: the pose consists of only chain "A"

In [8]:
print(pose.pdb_info())

PDB file name: 1AB1.clean.pdb
 Pose Range  Chain    PDB Range  |   #Residues         #Atoms

0001 -- 0046    A 0001  -- 0046  |   0046 residues;    00649 atoms
                           TOTAL |   0046 residues;    00649 atoms



 Although the pose contains only chain A, if the pose contained other chains we could make the selection to only select chain A and cysteine residues using an `AndResidueSelector`. We will use this ResidueSelector with `RestrictAbsentCanonicalAASRLT` (restrict absent canonical amino acids residue level task operation).

In [9]:
chain_A = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("A")
print(pyrosetta.rosetta.core.select.get_residues_from_subset(chain_A.apply(pose)))
chain_A_cys_res = pyrosetta.rosetta.core.select.residue_selector.AndResidueSelector(selector1=cys_res, selector2=chain_A)
print(pyrosetta.rosetta.core.select.get_residues_from_subset(chain_A_cys_res.apply(pose)))

vector1_unsigned_long[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]
vector1_unsigned_long[3, 4, 16, 26, 32, 40]


 Specify a `NotResidueSelector` selecting everything except the `chain_A_cys_res` ResidueSelector to use with a `RestrictToRepackingRLT` (restrict to repacking residue level task operation):

In [10]:
not_chain_A_cys_res = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(chain_A_cys_res)
print(pyrosetta.rosetta.core.select.get_residues_from_subset(not_chain_A_cys_res.apply(pose)))

vector1_unsigned_long[1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46]


 Quickly view the `not_chain_A_cys_res` ResidueSelector:

 Now we can set up the TaskOperations for `FastDesign`:

In [11]:
# The task factory accepts all the task operations
tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

# These are pretty standard
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.NoRepackDisulfides())


# Disable design (i.e. repack only) on not_chain_A_cys_res
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    pyrosetta.rosetta.core.pack.task.operation.RestrictToRepackingRLT(), not_chain_A_cys_res))

# Enable design on chain_A_cys_res
aa_to_design = pyrosetta.rosetta.core.pack.task.operation.RestrictAbsentCanonicalAASRLT()
aa_to_design.aas_to_keep("ADEFGHIKLMNPQRSTVWY")
tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
    aa_to_design, chain_A_cys_res))

# Convert the task factory into a PackerTask
packer_task = tf.create_task_and_apply_taskoperations(pose)
# View the PackerTask
print(packer_task)

core.pack.task: Packer task: initialize from command line()
#Packer_Task

Threads to request: ALL AVAILABLE

resid	pack?	design?	allowed_aas
1	TRUE	FALSE	THR:NtermProteinFull
2	TRUE	FALSE	THR
3	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
4	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
5	TRUE	FALSE	PRO
6	TRUE	FALSE	SER
7	TRUE	FALSE	ILE
8	TRUE	FALSE	VAL
9	TRUE	FALSE	ALA
10	TRUE	FALSE	ARG
11	TRUE	FALSE	SER
12	TRUE	FALSE	ASN
13	TRUE	FALSE	PHE
14	TRUE	FALSE	ASN
15	TRUE	FALSE	VAL
16	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
17	TRUE	FALSE	ARG
18	TRUE	FALSE	LEU
19	TRUE	FALSE	PRO
20	TRUE	FALSE	GLY
21	TRUE	FALSE	THR
22	TRUE	FALSE	SER
23	TRUE	FALSE	GLU
24	TRUE	FALSE	ALA
25	TRUE	FALSE	ILE
26	TRUE	TRUE	ALA,ASP,GLU,PHE,GLY,HIS,HIS_D,ILE,LYS,LEU,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
27	TRUE	FALSE	ALA
28	TRUE	FALSE	THR
29	TRUE	FALSE	TYR
30	TRUE	FALSE	THR
31	

 This time we will set up a `MoveMap` to establish which torsions are free to minimize

In [12]:
mm = pyrosetta.rosetta.core.kinematics.MoveMap()
mm.set_bb(True)
mm.set_chi(True)
mm.set_jump(True)

Set up `FastDesign`

In [13]:
rel_design = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn_in=scorefxn, standard_repeats=1, script_file="MonomerDesign2019")
rel_design.cartesian(True)
rel_design.set_task_factory(tf)
rel_design.set_movemap(mm)
rel_design.minimize_bond_angles(True)
rel_design.minimize_bond_lengths(True)

protocols.relax.RelaxScriptManager: Reading relax scripts list from database.
protocols.relax.RelaxScriptManager: Looking for MonomerDesign2019.txt
protocols.relax.RelaxScriptManager: ================== Reading script file: /usr/local/lib/python3.10/dist-packages/pyrosetta/database/sampling/relax_scripts/MonomerDesign2019.txt ==================
protocols.relax.RelaxScriptManager: repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: scale:fa_rep 0.059
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.092
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: scale:fa_rep 0.280
protocols.relax.RelaxScriptManager: repack
protocols.relax.RelaxScriptManager: scale:fa_rep 0.323
protocols.relax.RelaxScriptManager: min 0.01
protocols.relax.RelaxScriptManager: coord_cst_weight 0.0
protocols.relax.RelaxScriptMan

 Run the `FastDesign` mover. Note: this takes ~39.6 s

In [14]:
### BEGIN SOLUTION
%time rel_design.apply(pose)
### END SOLUTION

core.energy_methods.CartesianBondedEnergy: Creating new peptide-bonded energy container (46)
basic.io.database: Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: Binary rotamer library selected: /usr/local/lib/python3.10/dist-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: Using Dunbrack library binary file '/usr/local/lib/python3.10/dist-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: Dunbrack 2010 library took 0.434456 seconds to load from binary
protocols.relax.FastRelax: CMD: repeat  2280.6  0  0  0.55
protocols.relax.FastRelax: CMD: coord_cst_weight

 Inspect the resulting design!

## *Design Challenge*:

### Copy and modify this script to:
 - Repack all residues and allow all degrees of freedom to minimize (i.e. FastRelax) `start_pose` prior to downstream design. Use the resulting `start_pose` for analysis.
 - Re-design the cysteine residues in `start_pose` to allow only hydrophobic residues excluding glycine and proline
 - Only re-pack residues within a 6 Angstroms radius around the neighborhood of each cysteine residue (Cα-Cα distance)
   - Note this will require using:
   - The NeighborhoodResidueSelector: `pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(selector, distance, include_focus_in_subset)`
   - The prevent repacking residue level task operation: `pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT()`
 - Minimize only the side-chains (chi degrees of freedom) being re-packed or designed
 - Allow the backbone torsions to minimize, except for the first, last and any proline phi/psi angles

## *Extra Challenge!*

 Instead of using `pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset` TaskOperations, use `pyrosetta.rosetta.protocols.task_operations.ResfileCommandOperation` TaskOperations by applying resfile-style commands to ResidueSelectors. Example:
 > repack_only_task_operation = pyrosetta.rosetta.protocols.task_operations.ResfileCommandOperation()   
 > repack_only_task_operation.set_command("POLAR EMPTY NC R2 NC T6 NC OP5")   
 > repack_only_task_operation.set_residue_selector(selector)   
 > tf.push_back(repack_only_task_operation)

 By how many Angstroms RMSD did the backbone Cα atoms move?

In [23]:
from pyrosetta import init, pose_from_pdb, create_score_function
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.select.residue_selector import NeighborhoodResidueSelector, ChainSelector
from pyrosetta.rosetta.protocols.minimization_packing import MinMover
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.protocols.task_operations import ResfileCommandOperation

# Initialize PyRosetta
init()

# Load start_pose
start_pose = pose_from_pdb("1AB1.pdb")

# Repack all residues and minimize using FastRelax
fast_relax = FastRelax()
scorefxn = create_score_function("ref2015_cart.wts")
fast_relax.set_scorefxn(scorefxn)
fast_relax.max_iter(200)
fast_relax.apply(start_pose)

# Create a NeighborhoodResidueSelector for each cysteine residue
cysteine_selector = ChainSelector(1)  # Replace '1' with the chain ID of your CDRs

# Create a TaskFactory to control design/repacking
tf = TaskFactory()

# Set up a design task operation for hydrophobic residues excluding Glycine and Proline
design_task_operation = ResfileCommandOperation(cysteine_selector, "NATRO")# Use NATRO instead of HYDROPHOBIC
design_task_operation = ResfileCommandOperation()  # Set the command
design_task_operation.set_residue_selector(cysteine_selector)



PyRosetta-4 2023 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.01+release.00b79147e63be743438188f93a3f069ca75106d6 2023-12-25T16:35:48] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r366 2024.01+release.00b79147e63 00b79147e63be743438188f93a3f069ca75106d6 http://www.pyrosetta.org 2023-12-25T16:35:48
core.init: command: PyRosetta -ex1 -ex2aro -database /usr/local/lib/python3.10/dist-packages/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=77678179 seed_offset=0 real_seed=77678179
basic.random.init_random_generator: RandomGenerator:init: Normal mode, seed=77678179 RG_type=mt19937
core.import_pose.import_pose: File '1AB1.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromS

<!--NAVIGATION-->
< [Protein Design with a Resfile and FastRelax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [HBNet Before Design](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.05-HBNet-Before-Design.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.04-Protein-Design-2.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>